In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import svm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.svm import SVR

In [13]:
df = pd.read_csv("cleaned_data.csv")
df_t = pd.read_csv("cleane_test.csv")
df.head(5)

,Unnamed: 0,ID,Tag,Reputation,Answers,Username,Views,Upvotes,Tag_code
0,0,52664,a,3942.0,2.0,155623,7855.0,42.0,0
1,1,327662,a,26046.0,12.0,21781,55801.0,1175.0,0
2,2,468453,c,1358.0,4.0,56177,8067.0,60.0,1
3,3,96996,a,264.0,3.0,168793,27064.0,9.0,0
4,4,131465,c,4271.0,4.0,112223,13986.0,83.0,1


In [14]:
df_t.columns

Index(['Unnamed: 0', 'ID', 'Tag', 'Reputation', 'Answers', 'Username', 'Views',
       'Tag_code'],
      dtype='object')

In [15]:
rdf = df.drop(['Unnamed: 0', 'ID', 'Tag'],axis=1)
rtdf = df_t.drop(['Unnamed: 0', 'ID', 'Tag'],axis=1)

In [16]:
rdf.columns

Index(['Reputation', 'Answers', 'Username', 'Views', 'Upvotes', 'Tag_code'], dtype='object')

In [17]:
X_train,X_test,y_train,y_test = train_test_split(rdf[['Reputation', 'Answers', 'Views','Tag_code']],rdf['Upvotes'],test_size=0.10, random_state=42)

In [24]:
model1 = GradientBoostingRegressor(random_state=4)
model2 = RandomForestRegressor(random_state=3)
model3 = SVR()

In [25]:
from sklearn.ensemble import VotingClassifier

In [ ]:
#model1.fit(rdf[['Reputation', 'Answers', 'Views','Tag_code']],rdf['Upvotes'])
#model2.fit(rdf[['Reputation', 'Answers', 'Views','Tag_code']],rdf['Upvotes'])
model3.fit(rdf[['Reputation', 'Answers', 'Views','Tag_code']],rdf['Upvotes'])

C:\Users\Divay Mohan\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

clf = make_pipeline(StandardScaler(), SVC())

In [ ]:
model3.fit(rdf[['Reputation', 'Answers', 'Views','Tag_code']],rdf['Upvotes'])

In [ ]:
pred = model3.predict(rtdf)
pred

In [ ]:
result = pd.DataFrame()

In [ ]:
result["ID"] = df_t["ID"]
result["Upvotes"] = pred

In [ ]:
result.head()

In [ ]:
result.to_csv("sub1.csv",index=False)

In [82]:
from sklearn.ensemble import VotingRegressor

In [83]:
ereg = VotingRegressor(estimators=[('gb', model1), ('rf', model2)])

In [84]:
ereg.fit(X_train,y_train)

VotingRegressor(estimators=[('gb',
                             GradientBoostingRegressor(alpha=0.9,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                                   